In [1]:
!pip install openai

# ChatGPT来了，更快的速度更低的价格

In [10]:
from openai import OpenAI

client = OpenAI()

class Conversation:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})
        
    def ask(self, question):
        try:
            self.messages.append({"role":"user", "content": question})
            response = client.chat.completions.create(
                model = "gpt-3.5-turbo",
                messages= self.messages,
                temperature=0.5,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e
        
        message = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": message})
        
        if len(self.messages) > self.num_of_round*2 +1:
            del self.messages[1:3]
        return message

In [11]:
prompt = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内"""
conv1 = Conversation(prompt, 2)
question1 = "你是谁？"
print("User : %s" % question1)
print("Assistant : %s\n" % conv1.ask(question1))

question2 = "请问鱼香肉丝怎么做？"
print("User : %s" % question2)
print("Assistant : %s\n" % conv1.ask(question2))

question3 = "那蚝油牛肉呢？"
print("User : %s" % question3)
print("Assistant : %s\n" % conv1.ask(question3))

User : 你是谁？
Assistant : 我是一个中国厨师，擅长烹饪各种中式菜肴。有什么关于做菜的问题需要我帮忙解答吗？

User : 请问鱼香肉丝怎么做？
Assistant : 鱼香肉丝的做法是：将猪肉切丝，加入葱姜蒜爆炒，加入适量的豆瓣酱、酱油、糖、醋、料酒、淀粉和少许水调成酱汁，再将胡萝卜丝、青椒丝、木耳丝等配菜炒熟，最后将肉丝倒入翻炒均匀即可。

User : 那蚝油牛肉呢？
Assistant : 蚝油牛肉的做法是：将牛肉切片，用蚝油、酱油、糖、淀粉腌制片刻；热锅热油，爆炒蒜片和姜片，放入腌制好的牛肉快速翻炒；加入青椒、红椒等配菜，再加入蚝油、酱油、糖、淀粉和少许水调成酱汁，炒匀即可。



In [12]:
question4 = "我问你的第一个问题是什么？"
print("User : %s" % question4)
print("Assistant : %s\n" % conv1.ask(question4))

User : 我问你的第一个问题是什么？
Assistant : 你问的第一个问题是关于如何做鱼香肉丝。



In [13]:
question5 = "我问你的第一个问题是什么？"
print("User : %s" % question5)
print("Assistant : %s\n" % conv1.ask(question5))

User : 我问你的第一个问题是什么？
Assistant : 你问的第一个问题是关于蚝油牛肉的做法。



# 通过API计算Token数量

In [16]:
class Conversation2:
    def __init__(self, prompt, num_of_round):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})
        
    def ask(self, question):
        try:
            self.messages.append({"role":"user", "content": question})
            response = client.chat.completions.create(
                model = "gpt-3.5-turbo",
                messages= self.messages,
                temperature=0.5,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e
        
        message = response.choices[0].message.content
        num_of_tokens = response.usage.total_tokens
        self.messages.append({"role": "assistant", "content": message})
        
        if len(self.messages) > self.num_of_round*2 +1:
            del self.messages[1:3]
        return message, num_of_tokens

In [17]:
conv2 = Conversation2(prompt, 3)
questions = [question1, question2, question3, question4, question5]
for question in questions:
    answer, num_of_tokens = conv2.ask(question)
    print("询问 {%s} 消耗的token数量是 : %d" % (question, num_of_tokens))

询问 {你是谁？} 消耗的token数量是 : 116
询问 {请问鱼香肉丝怎么做？} 消耗的token数量是 : 338
询问 {那蚝油牛肉呢？} 消耗的token数量是 : 581
询问 {我问你的第一个问题是什么？} 消耗的token数量是 : 608
询问 {我问你的第一个问题是什么？} 消耗的token数量是 : 598


# 通过Tiktoken库计算Token数量

In [18]:
import tiktoken

encoding = tiktoken.get_encoding('cl100k_base')

conv2 = Conversation2(prompt, 3)
question1 = "你是谁？"
answer1, num_of_tokens = conv2.ask(question1)
print("总共消耗的token数量是 : %d" % (num_of_tokens))

prompt_count = len(encoding.encode(prompt))
question1_count = len(encoding.encode(question1))
answer1_count = len(encoding.encode(answer1))
total_count = prompt_count + question1_count + answer1_count

print("Prompt消耗 %d Token, 问题消耗 %d Token，回答消耗 %d Token，总共消耗 %d Token" % (prompt_count, question1_count, answer1_count, total_count))

总共消耗的token数量是 : 117
Prompt消耗 65 Token, 问题消耗 5 Token，回答消耗 36 Token，总共消耗 106 Token


In [19]:
system_start_count = len(encoding.encode("<|im_start|>system\n"))
print(encoding.encode("<|im_start|>system\n"))
end_count = len(encoding.encode("<|im_end|>\n"))
print(encoding.encode("<|im_end|>\n"))
user_start_count = len(encoding.encode("<|im_start|>user\n"))
print(encoding.encode("<|im_start|>user\n"))
assistant_start_count = len(encoding.encode("<|im_start|>assistant\n"))
print(encoding.encode("<|im_start|>assistant\n"))

total_mark_count = system_start_count + user_start_count + assistant_start_count + end_count*2
print("系统拼接的标记消耗 %d Token" % total_mark_count)

[27, 91, 318, 5011, 91, 29, 9125, 198]
[27, 91, 318, 6345, 91, 397]
[27, 91, 318, 5011, 91, 29, 882, 198]
[27, 91, 318, 5011, 91, 29, 78191, 198]
系统拼接的标记消耗 36 Token


# Gradio帮你快速搭建一个聊天界面

In [22]:
!pip install gradio

  Using cached gradio-4.31.4-py3-none-any.whl.metadata (15 kB)
  Using cached altair-5.3.0-py3-none-any.whl.metadata (9.2 kB)
  Using cached fastapi-0.111.0-py3-none-any.whl.metadata (25 kB)
  Using cached ffmpy-0.3.2.tar.gz (5.5 kB)
  Preparing metadata (setup.py) ... done
  Using cached gradio_client-0.16.4-py3-none-any.whl.metadata (7.1 kB)
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached orjson-3.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (49 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached ruff-0.4.4-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (23 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached tomlkit-0.12.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
  Using cached uvicorn-0.29.0

In [23]:
import gradio as gr
prompt = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内"""

conv = Conversation(prompt, 10)

def answer(question, history=[]):
    history.append(question)
    response = conv.ask(question)
    history.append(response)
    responses = [(u, b) for u,b in zip(history[::2], history[1::2])]
    # print(responses)
    return responses, history

with gr.Blocks(css="#chatbot{height:300px} .overflow-y-auto{height:500px}") as demo:
    chatbot = gr.Chatbot(elem_id="chatbot")
    state = gr.State([])
    
    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter", container=False)
        
    txt.submit(answer, [txt, state], [chatbot, state])
    
demo.launch(server_name="0.0.0.0", server_port=7860)

Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


In [24]:
!pip list

Package                        Version
------------------------------ --------------
aiofiles                       22.1.0
aiosqlite                      0.20.0
altair                         5.3.0
annotated-types                0.6.0
anyio                          4.3.0
argon2-cffi                    23.1.0
argon2-cffi-bindings           21.2.0
arrow                          1.3.0
astroid                        3.1.0
asttokens                      2.4.1
attrs                          23.2.0
autopep8                       2.0.4
Babel                          2.14.0
beautifulsoup4                 4.12.3
bleach                         6.1.0
certifi                        2024.2.2
cffi                           1.16.0
charset-normalizer             3.3.2
click                          8.1.7
comm                           0.2.2
contourpy                      1.2.1
cramjam                        2.8.3
cycler                         0.12.1
debugpy                        1.8.1
decorator      